Duplicate images into new folder

In [1]:
import os
import shutil

# Input image path (relative path)
input_image_path = "dtd_folder/dtd/images/honeycombed/honeycombed_0003.jpg"  # Replace with your image path

# Output directory
output_dir = "dtd_folder/dtd/images/curatedHoneycomb"
os.makedirs(output_dir, exist_ok=True)  # Create the folder if it doesn't exist

# Number of duplicates
num_duplicates = 120

# Loop to create duplicates with unique filenames
for i in range(1, num_duplicates + 1):
    output_file = os.path.join(output_dir, f"honeycomb_{i}.jpg")
    shutil.copy(input_image_path, output_file)

print(f"Duplicated the image {num_duplicates} times in the folder '{output_dir}'.")

Duplicated the image 120 times in the folder 'dtd_folder/dtd/images/curatedHoneycomb'.


In [2]:
import cv2
import homography
# Input image path (relative path)
input_dir = "dtd_folder/dtd/images/curatedHoneycomb"

# Output directory
output_dir = "dtd_folder/dtd/images/curatedHoneycombHOMOGRAPHY"
os.makedirs(output_dir, exist_ok=True)  # Create the folder if it doesn't exist

# Loop to create duplicates with unique filenames
for i in range(1, 121):
    output_file = os.path.join(output_dir, f"honeycomb_{i}.jpg")
    input_file = os.path.join(input_dir, f'honeycomb_{i}.jpg')
    img = cv2.imread(input_file)
    H = homography.generate_homography_warp(img.shape[:2])
    img_out = homography.apply_homography(img, H)
    cv2.imwrite(output_file, img_out)
    
print(f"Duplicated the image {num_duplicates} times in the folder '{output_dir}'.")

Duplicated the image 120 times in the folder 'dtd_folder/dtd/images/curatedHoneycombHOMOGRAPHY'.


In [3]:
from utils import load_and_preprocess_images
from config import Config
config = Config()

images_dir = 'C:/Users/ASC_Student/Documents/GitHub/GAN-Texture-Synthesis/dtd_folder/dtd/images/curatedHoneycomb'
images_dir_warped = 'C:/Users/ASC_Student/Documents/GitHub/GAN-Texture-Synthesis/dtd_folder/dtd/images/curatedHoneycombHOMOGRAPHY'
print("Loading images!")

dataset = load_and_preprocess_images(images_dir, target_size=(160, 160), batch_size=config.batch_size)
dataset_warped = load_and_preprocess_images(images_dir_warped, target_size=(160, 160), batch_size=config.batch_size)

combined_dataset = dataset.concatenate(dataset_warped)

Loading images!
Found 120 files.
Found 120 files.


In [ ]:
from trainWGAN import train 

print("Starting PSGAN training...wish me luck")
train(dataset, epochs=config.epoch_count)
print("Training complete! Check 'samples/' for generated images and 'models/' for saved models.")

Generated output shape: (1, 160, 160, 3)
Starting PSGAN training...wish me luck
Epoch 1/1000
Step 0: Generator Loss: 0.1010, Disciminator loss: 8.1669
Step 1: Generator Loss: 0.0330, Disciminator loss: 8.2088
Step 2: Generator Loss: 0.1435, Disciminator loss: 7.9055
Step 3: Generator Loss: -0.0007, Disciminator loss: 7.8943
Step 4: Generator Loss: 0.0577, Disciminator loss: 7.8367
Step 5: Generator Loss: 0.0087, Disciminator loss: 7.9012
Step 6: Generator Loss: 0.0445, Disciminator loss: 7.7336
Step 7: Generator Loss: -0.0449, Disciminator loss: 7.5364
Step 8: Generator Loss: 0.0568, Disciminator loss: 7.5426
Step 9: Generator Loss: 0.0870, Disciminator loss: 7.1773
Step 10: Generator Loss: -0.1057, Disciminator loss: 7.2220
Step 11: Generator Loss: -0.0226, Disciminator loss: 7.1014
Step 12: Generator Loss: 0.0649, Disciminator loss: 6.9559
Step 13: Generator Loss: -0.0715, Disciminator loss: 6.6814
Step 14: Generator Loss: -0.2010, Disciminator loss: 6.6743
Step 15: Generator Loss: -